# J-Quants API V2 Quick Startガイド

In [ ]:
#@title 初期設定＆各種import
#@markdown ←にある▶ボタンを押すとGoogle Colabでコードを実行することができます。

#@markdown まずはこちらのコードを実行し、J-Quants APIを利用するため必要なパッケージをimportしましょう。

import json
import sys
import requests

from IPython.display import display
import pandas as pd

pd.set_option("display.max_columns", None)

API_URL = "https://api.jquants.com/v2"

## Step1：API利用開始までの流れ　※初回のみ実施

J-Quants APIのご利用を検討いただき、ありがとうございます。

**J-Quants APIをご利用いただくには、以下の2つを事前に行っていただく必要がございます。**
 1. [J-Quants Webサイト](https://jpx-jquants.com/)への登録
 2. J-Quants API利用のためのプラン（Free, Light, Standard, Premium）選択

まだ、ご登録もしくはプラン選択がお済みでない方は、まず上記の2項目を行っていただきますようお願いいたします。  
より具体的な手順は[こちら](https://jpx-jquants.com/ja/spec/quickstart)をご参照ください。

## Step2：APIキー取得

V2ではAPIキー方式による認証を採用しています。

APIキーはJ-Quants Webサイトのダッシュボードページから取得できます。

1. [J-Quants Webサイト](https://jpx-jquants.com/)にログイン
2. ダッシュボードページにアクセス
3. APIキーを取得（APIキーに有効期限はありません）

In [ ]:
#@title **APIキーを設定**
#@markdown J-Quants WebサイトのダッシュボードにてAPIキーを取得し、以下に貼り付けてください。

api_key = ""#@param {type: "string"}

if api_key:
    headers = {"x-api-key": api_key}
    print("APIキーが設定されました。J-Quants APIを利用する準備が完了しました。")
else:
    print("APIキーを入力してください。")

## Step3：取得したAPIキーを用いて各APIをご利用ください。

### Freeプラン以上のプランで利用できるAPI
- 上場銘柄一覧（/equities/master）
- 株価四本値*（/equities/bars/daily）
- 財務情報（/fins/summary）
- 決算発表予定日（/equities/earnings-calendar）
- 取引カレンダー（/markets/calendar）

\* プレミアムプランのユーザのみ、前後場の四本値及び取引高・取引代金の情報が取得可能


In [ ]:
#@title 上場銘柄一覧（/equities/master）

#@markdown - 過去時点での銘柄情報、当日の銘柄情報および翌営業日時点の銘柄情報が取得可能です。
#@markdown - データの取得では、銘柄コード（code）または日付（date）の指定が可能です。

#@markdown （データ更新時刻）
#@markdown - 毎営業日の17:30頃、翌営業日の8:00頃

code = ""#@param {type:"string"}
date = ""#@param {type:"string"}

params = {}
if code != "":
  params["code"] = code
if date != "":
  params["date"] = date

res = requests.get(f"{API_URL}/equities/master", params=params, headers=headers)

if res.status_code == 200:
  d = res.json()
  data = d["data"]
  while "pagination_key" in d:
    params["pagination_key"] = d["pagination_key"]
    res = requests.get(f"{API_URL}/equities/master", params=params, headers=headers)
    d = res.json()
    data += d["data"]
  df = pd.DataFrame(data)
  display(df)
else:
  print(res.json())

In [ ]:
#@title 株価四本値（/equities/bars/daily）

#@markdown - 株価は分割・併合を考慮した調整済み株価（小数点第２位四捨五入）と調整前の株価を取得することができます。
#@markdown - データの取得では、銘柄コード（code）または日付（date）の指定が必須となります。

#@markdown （データ更新時刻）
#@markdown - 毎営業日の17:00頃

#@markdown - Premiumプランの方には、日通しに加え、前場(Morning)及び後場(Afternoon)の四本値及び取引高（調整前・後両方）・取引代金が取得可能です。


code = ""#@param {type:"string"}
date = ""#@param {type:"string"}
from_ = "" #@param {type:"string"}
to = "" #@param {type:"string"}

params = {}
if code != "":
  params["code"] = code
if date != "":
  params["date"] = date
if from_ != "":
  params["from"] = from_
if to != "":
  params["to"] = to

res = requests.get(f"{API_URL}/equities/bars/daily", params=params, headers=headers)

if res.status_code == 200:
  d = res.json()
  data = d["data"]
  while "pagination_key" in d:
    params["pagination_key"] = d["pagination_key"]
    res = requests.get(f"{API_URL}/equities/bars/daily", params=params, headers=headers)
    d = res.json()
    data += d["data"]
  df = pd.DataFrame(data)
  display(df)
else:
  print(res.json())

In [ ]:
#@title 財務情報（/fins/summary）

#@markdown - 財務情報APIでは、上場企業がTDnetへ提出する決算短信Summary等を基に作成された、四半期毎の財務情報を取得することができます。
#@markdown - データの取得では、銘柄コード（code）または開示日（date）の指定が必須です。

#@markdown （データ更新時刻）
#@markdown - 速報18:00頃、確報24:30頃


code = ""#@param {type:"string"}
date = ""#@param {type:"string"}

params = {}
if code != "":
  params["code"] = code
if date != "":
  params["date"] = date

res = requests.get(f"{API_URL}/fins/summary", params=params, headers=headers)

if res.status_code == 200:
  d = res.json()
  data = d["data"]
  while "pagination_key" in d:
    params["pagination_key"] = d["pagination_key"]
    res = requests.get(f"{API_URL}/fins/summary", params=params, headers=headers)
    d = res.json()
    data += d["data"]
  df = pd.DataFrame(data)
  display(df)
else:
  print(res.json())

In [ ]:
#@title 決算発表予定日（/equities/earnings-calendar）

#@markdown （データ更新時刻）
#@markdown - 不定期（更新がある日は）19:00頃

#@markdown - [当該ページ](https://www.jpx.co.jp/listing/event-schedules/financial-announcement/index.html)で、3月期・９月期決算会社分に更新があった場合のみ19時ごろに更新されます。

params = {}

res = requests.get(f"{API_URL}/equities/earnings-calendar", params=params, headers=headers)

if res.status_code == 200:
  d = res.json()
  data = d["data"]
  while "pagination_key" in d:
    params["pagination_key"] = d["pagination_key"]
    res = requests.get(f"{API_URL}/equities/earnings-calendar", params=params, headers=headers)
    d = res.json()
    data += d["data"]
  df = pd.DataFrame(data)
  display(df)
else:
  print(res.json())

In [ ]:
#@title 取引カレンダー（/markets/calendar）

#@markdown - 東証およびOSEにおける営業日、休業日、ならびにOSEにおける祝日取引の有無の情報を取得できます。
#@markdown - データの取得では、休日区分（hol_div）または日付（from/to）の指定が可能です。

#@markdown （データ更新日）
#@markdown - 不定期（原則として、毎年2月頃をめどに翌年1年間の営業日および祝日取引実施日（予定）を更新します。）


hol_div = ""#@param ["0", "1", "2", "3"]{allow-input:true}
from_ = "" #@param {type:"string"}
to = "" #@param {type:"string"}

params = {}
if hol_div != "":
  params["hol_div"] = hol_div
if from_ != "":
  params["from"] = from_
if to != "":
  params["to"] = to

res = requests.get(f"{API_URL}/markets/calendar", params=params, headers=headers)

if res.status_code == 200:
  d = res.json()
  data = d["data"]
  while "pagination_key" in d:
    params["pagination_key"] = d["pagination_key"]
    res = requests.get(f"{API_URL}/markets/calendar", params=params, headers=headers)
    d = res.json()
    data += d["data"]
  df = pd.DataFrame(data)
  display(df)
else:
  print(res.json())

### Lightプラン以上のプランで利用できるAPI
- 投資部門別情報（/equities/investor-types）
- TOPIX四本値（/indices/bars/daily/topix）

In [ ]:
#@title 投資部門別（株式）データ（/equities/investor-types）

#@markdown - 投資部門別売買状況（金額）のデータを取得することができます。
#@markdown - 投資部門別売買状況は、個人・外国人・金融機関など、投資家ごとの売買動向をまとめた情報です。
#@markdown - 基本的には[こちら](https://www.jpx.co.jp/markets/statistics-equities/investor-type/index.html)のページで掲載しているものと同等のものになります。
#@markdown - データの取得では、セクション（section）または日付（from/to）の指定が可能です。

#@markdown （データ更新時刻）
#@markdown - 原則、毎週第４営業日18:00頃


section = ""#@param ["TSE1st", "TSE2nd", "TSEMothers", "TSEJASDAQ", "TSEPrime", "TSEStandard", "TSEGrowth", "TokyoNagoya"] {allow-input: true}
from_ = "" #@param {type:"string"}
to = "" #@param {type:"string"}

params = {}
if section != "":
  params["section"] = section
if from_ != "":
  params["from"] = from_
if to != "":
  params["to"] = to

res = requests.get(f"{API_URL}/equities/investor-types", params=params, headers=headers)

if res.status_code == 200:
  d = res.json()
  data = d["data"]
  while "pagination_key" in d:
    params["pagination_key"] = d["pagination_key"]
    res = requests.get(f"{API_URL}/equities/investor-types", params=params, headers=headers)
    d = res.json()
    data += d["data"]
  df = pd.DataFrame(data)
  display(df)
else:
  print(res.json())


In [ ]:
#@title TOPIX四本値（/indices/bars/daily/topix）

#@markdown - 日次のTOPIX指数の四本値について取得することができます。
#@markdown - 日付（from/to）の指定が可能です

#@markdown （データ更新時刻）
#@markdown - 毎営業日の16:30頃


date_from = ""#@param {type:"string"}
date_to = ""#@param {type:"string"}

params = {}

if date_from != "":
  params["from"] = date_from
if date_to != "":
  params["to"] = date_to

res = requests.get(f"{API_URL}/indices/bars/daily/topix", params=params, headers=headers)

if res.status_code == 200:
  d = res.json()
  data = d["data"]
  while "pagination_key" in d:
    params["pagination_key"] = d["pagination_key"]
    res = requests.get(f"{API_URL}/indices/bars/daily/topix", params=params, headers=headers)
    d = res.json()
    data += d["data"]
  df = pd.DataFrame(data)
  display(df)
else:
  print(res.json())


### Standard以上のプランで利用できるAPI
- 指数四本値（/indices/bars/daily）
- 日経225オプション四本値（/derivatives/bars/daily/options/225）
- 信用取引週末残高（/markets/margin-interest）
- 業種別空売り比率（/markets/short-ratio）
- 空売り残高報告（/markets/short-sale-report）
- 日々公表信用取引残高（/markets/margin-alert）

In [ ]:
#@title 指数四本値（/indices/bars/daily）

#@markdown - 日次の各種指数の四本値について取得することができます。
#@markdown - データの取得では、指数コード（code）または日付（date）の指定が必須となります。

#@markdown （データ更新時刻）
#@markdown - 毎営業日の16:30頃


code = ""#@param {type:"string"}
date = ""#@param {type:"string"}
from_ = "" #@param {type:"string"}
to = "" #@param {type:"string"}

params = {}
if code != "":
  params["code"] = code
if date != "":
  params["date"] = date
if from_ != "":
  params["from"] = from_
if to != "":
  params["to"] = to

res = requests.get(f"{API_URL}/indices/bars/daily", params=params, headers=headers)

if res.status_code == 200:
  d = res.json()
  data = d["data"]
  while "pagination_key" in d:
    params["pagination_key"] = d["pagination_key"]
    res = requests.get(f"{API_URL}/indices/bars/daily", params=params, headers=headers)
    d = res.json()
    data += d["data"]
  df = pd.DataFrame(data)
  display(df)
else:
  print(res.json())

In [ ]:
#@title 日経225オプション四本値（/derivatives/bars/daily/options/225）

#@markdown - 日次の日経225指数オプションの四本値や売買高、清算値段等について取得することができます。
#@markdown - 日付（date）の指定が必須です

#@markdown （データ更新時刻）
#@markdown - 毎営業日の27:00頃


date = "" #@param {type:"string"}
params = {}
if date != "":
  params["date"] = date

res = requests.get(f"{API_URL}/derivatives/bars/daily/options/225", params=params, headers=headers)

if res.status_code == 200:
  d = res.json()
  data = d["data"]
  while "pagination_key" in d:
    params["pagination_key"] = d["pagination_key"]
    res = requests.get(f"{API_URL}/derivatives/bars/daily/options/225", params=params, headers=headers)
    d = res.json()
    data += d["data"]
  df = pd.DataFrame(data)
  display(df)
else:
  print(res.json())

In [ ]:
#@title 信用取引週末残高（/markets/margin-interest）

#@markdown - 制度・一般信用取引における各銘柄の前週末の残高についてデータを取得することができます。
#@markdown - 本データは[こちら](https://www.jpx.co.jp/markets/statistics-equities/margin/index.html)の個別銘柄信用取引残高表のデータをヒストリカルで提供するものになります。
#@markdown - 銘柄コード（code）もしくは日付（date）の指定が必須です。
#@markdown （From, Toはcodeを入れている場合にのみ可能）

#@markdown （データ更新時刻）
#@markdown - 原則毎週第２営業日の16:30頃


code = ""#@param {type:"string"}
date = ""#@param {type:"string"}
from_ = "" #@param {type:"string"}
to = "" #@param {type:"string"}

params = {}
if code != "":
  params["code"] = code
if date != "":
  params["date"] = date
if from_ != "":
  params["from"] = from_
if to != "":
  params["to"] = to

res = requests.get(f"{API_URL}/markets/margin-interest", params=params, headers=headers)

if res.status_code == 200:
  d = res.json()
  data = d["data"]
  while "pagination_key" in d:
    params["pagination_key"] = d["pagination_key"]
    res = requests.get(f"{API_URL}/markets/margin-interest", params=params, headers=headers)
    d = res.json()
    data += d["data"]
  df = pd.DataFrame(data)
  display(df)
else:
  print(res.json())

In [ ]:
#@title 業種別空売り比率（/markets/short-ratio）

#@markdown - 日次の３３業種別の空売りの売買代金について取得することができます。
#@markdown - ３３業種コード（s33）もしくは日付（date）の指定が必須です。
#@markdown （From, Toはs33を入れている場合にのみ可能）

#@markdown （データ更新時刻）
#@markdown - 毎営業日の16:30頃

#@markdown （その他留意点）
#@markdown - 業種コード9999は、ETFやREIT等の３３業種に含まれない銘柄のものになります。


s33 = ""#@param {type:"string"}
date = ""#@param {type:"string"}
from_ = "" #@param {type:"string"}
to = "" #@param {type:"string"}

params = {}
if s33 != "":
  params["s33"] = s33
if date != "":
  params["date"] = date
if from_ != "":
  params["from"] = from_
if to != "":
  params["to"] = to

res = requests.get(f"{API_URL}/markets/short-ratio", params=params, headers=headers)

if res.status_code == 200:
  d = res.json()
  data = d["data"]
  while "pagination_key" in d:
    params["pagination_key"] = d["pagination_key"]
    res = requests.get(f"{API_URL}/markets/short-ratio", params=params, headers=headers)
    d = res.json()
    data += d["data"]
  df = pd.DataFrame(data)
  display(df)
else:
  print(res.json())

In [ ]:
#@title 空売り残高報告（/markets/short-sale-report）

#@markdown - 「有価証券の取引等の規制に関する内閣府令」に基づき、取引参加者より報告を受けたもののうち、残高割合が0.5％以上のものについての情報を取得できます。
#@markdown - データの取得では、銘柄コード（code）、公表日（disc_date）、計算日（calc_date）のいずれかの指定が必須となります。

#@markdown （データ更新時刻）
#@markdown - 毎営業日の17:30頃

code = ""#@param {type:"string"}
disc_date = ""#@param {type:"string"}
disc_date_from = ""#@param {type:"string"}
disc_date_to = ""#@param {type:"string"}
calc_date = ""#@param {type:"string"}

params = {}
if code != "":
  params["code"] = code
if disc_date != "":
  params["disc_date"] = disc_date
if disc_date_from != "":
  params["disc_date_from"] = disc_date_from
if disc_date_to != "":
  params["disc_date_to"] = disc_date_to
if calc_date != "":
  params["calc_date"] = calc_date

res = requests.get(f"{API_URL}/markets/short-sale-report", params=params, headers=headers)

if res.status_code == 200:
  d = res.json()
  data = d["data"]
  while "pagination_key" in d:
    params["pagination_key"] = d["pagination_key"]
    res = requests.get(f"{API_URL}/markets/short-sale-report", params=params, headers=headers)
    d = res.json()
    data += d["data"]
  df = pd.DataFrame(data)
  display(df)
else:
  print(res.json())

In [ ]:
#@title 日々公表信用取引残高（/markets/margin-alert）

#@markdown - 東京証券取引所または日本証券金融が、日次の信用取引残高を公表する必要があると認めた銘柄のみが収録されます。
#@markdown - データの取得では、銘柄コード（code）、公表日（date）のいずれかの指定が必須となります。

#@markdown （データ更新時刻）
#@markdown - 毎営業日の16:30頃

code = ""#@param {type:"string"}
_date = ""#@param {type:"string"}
_from = ""#@param {type:"string"}
_to = ""#@param {type:"string"}

params = {}
if code != "":
  params["code"] = code
if _date != "":
  params["date"] = _date
if _from != "":
  params["from"] = _from
if _to != "":
  params["to"] = _to

res = requests.get(f"{API_URL}/markets/margin-alert", params=params, headers=headers)

if res.status_code == 200:
  d = res.json()
  data = d["data"]
  while "pagination_key" in d:
    params["pagination_key"] = d["pagination_key"]
    res = requests.get(f"{API_URL}/markets/margin-alert", params=params, headers=headers)
    d = res.json()
    data += d["data"]
  df = pd.DataFrame(data)
  display(df)
else:
  print(res.json())

### Premiumプランで利用できるAPI
- 売買内訳データ（/markets/breakdown）
- 前場四本値（/equities/bars/daily/am）
- 配当金情報（/fins/dividend）
- 財務諸表(BS/PL/CF)（/fins/details）
- 先物四本値（/derivatives/bars/daily/futures）
- オプション四本値（/derivatives/bars/daily/options）

In [ ]:
#@title 売買内訳データ（/markets/breakdown）

#@markdown - 売買内訳データは、東証上場銘柄の東証市場における銘柄別の日次売買代金・売買高(立会内取引に限る)について、信用取引や空売りの利用に関して、発注時のフラグ情報を用いて細分化したデータです。
#@markdown - 銘柄コード（code）もしくは日付（date）の指定が必須です。
#@markdown （From, Toはcodeを入れている場合にのみ可能）

#@markdown （データ更新時刻）
#@markdown - 毎営業日の18:00頃

#@markdown （その他留意点）
#@markdown - 当日に立会内取引が成立しなかった場合（約定なし）、当該銘柄はレコードに含まれていません。

code = ""#@param {type:"string"}
date = ""#@param {type:"string"}
from_ = "" #@param {type:"string"}
to = "" #@param {type:"string"}

params = {}
if code != "":
  params["code"] = code
if date != "":
  params["date"] = date
if from_ != "":
  params["from"] = from_
if to != "":
  params["to"] = to

res = requests.get(f"{API_URL}/markets/breakdown", params=params, headers=headers)

if res.status_code == 200:
  d = res.json()
  data = d["data"]
  while "pagination_key" in d:
    params["pagination_key"] = d["pagination_key"]
    res = requests.get(f"{API_URL}/markets/breakdown", params=params, headers=headers)
    d = res.json()
    data += d["data"]
  df = pd.DataFrame(data)
  display(df)
else:
  print(res.json())

In [ ]:
#@title 前場四本値（/equities/bars/daily/am）

#@markdown - 各銘柄の前場の四本値及び取引高・代金について、当日の前場終了後のデータを取得できます。
#@markdown - 銘柄コード（code）を指定することができます。codeパラメータがない場合は、全銘柄取得されます。

#@markdown （データ更新時刻）
#@markdown - 毎営業日の12:00頃

#@markdown （その他留意点）
#@markdown - 本APIのデータは当日12:00頃〜翌日の朝6:00頃まで取得可能です。
#@markdown - 当日以外のヒストリカルのデータは、Premiumユーザの方は株価四本値（/equities/bars/daily）のAPIで取得可能です。
#@markdown - なお、上記以外の時間にAPIコールした場合は、StatusCode = 210が返却されます。（通常は200）


code = ""#@param {type:"string"}

params = {}
if code != "":
  params["code"] = code

res = requests.get(f"{API_URL}/equities/bars/daily/am", params=params, headers=headers)

if res.status_code == 200:
  d = res.json()
  data = d["data"]
  while "pagination_key" in d:
    params["pagination_key"] = d["pagination_key"]
    res = requests.get(f"{API_URL}/equities/bars/daily/am", params=params, headers=headers)
    d = res.json()
    data += d["data"]
  df = pd.DataFrame(data)
  display(df)
else:
  print(res.json())

In [ ]:
#@title 配当金情報（/fins/dividend）

#@markdown - 上場会社の配当（決定・予想）に関する１株当たり配当金額、基準日、権利落日及び支払開始予定日等の情報が取得できます。
#@markdown - 銘柄コード（code）もしくは日付（date）の指定が必須です。
#@markdown （From, Toはcodeを入れている場合にのみ可能）

#@markdown （データ更新時刻）
#@markdown - 毎営業日の19:00頃


code = ""#@param {type:"string"}
date = ""#@param {type:"string"}
from_ = "" #@param {type:"string"}
to = "" #@param {type:"string"}

params = {}
if code != "":
  params["code"] = code
if date != "":
  params["date"] = date
if from_ != "":
  params["from"] = from_
if to != "":
  params["to"] = to

res = requests.get(f"{API_URL}/fins/dividend", params=params, headers=headers)

if res.status_code == 200:
  d = res.json()
  data = d["data"]
  while "pagination_key" in d:
    params["pagination_key"] = d["pagination_key"]
    res = requests.get(f"{API_URL}/fins/dividend", params=params, headers=headers)
    d = res.json()
    data += d["data"]
  df = pd.DataFrame(data)
  display(df)
else:
  print(res.json())

In [ ]:
#@title 財務諸表(BS/PL/CF)（/fins/details）

#@markdown - 財務諸表(BS/PL/CF)APIでは、上場企業の四半期毎の財務情報における、貸借対照表、損益計算書に記載の項目を取得することができます。
#@markdown - 本コードでは、出力結果の表示方法をformat_typeを指定して切り替え可能です。flat:全項目をヘッダーに表示する、non-flat:詳細項目を1列に集約します。
#@markdown - データの取得では、銘柄コード（code）または開示日（date）の指定が必須です。

#@markdown （データ更新時刻）
#@markdown - 速報18:00頃、確報24:30頃


format_type = "non-flat"#@param ["non-flat", "flat"]
code = ""#@param {type:"string"}
date = ""#@param {type:"string"}

params = {}
if code != "":
  params["code"] = code
if date != "":
  params["date"] = date

res = requests.get(f"{API_URL}/fins/details", params=params, headers=headers)

if res.status_code == 200:
  d = res.json()
  data = d["data"]
  while "pagination_key" in d:
    params["pagination_key"] = d["pagination_key"]
    res = requests.get(f"{API_URL}/fins/details", params=params, headers=headers)
    d = res.json()
    data += d["data"]
  if format_type == "non-flat":
    df = pd.DataFrame(data)
  else:
    df = pd.json_normalize(data)
  display(df)
else:
  print(res.json())

In [ ]:
#@title 先物四本値（/derivatives/bars/daily/futures）

#@markdown - 先物四本値APIでは、先物に関する、四本値や清算値段、理論価格に関する情報を取得することができます。
#@markdown - データの取得では、日付（date）の指定が必須です。

#@markdown （データ更新時刻）
#@markdown - 毎営業日の27:00頃

category = ""#@param {type:"string"}
date = ""#@param {type:"string"}
contract_flag = ""#@param{type:"string"}

params = {}
if category != "":
  params["category"] = category
if date != "":
  params["date"] = date
if contract_flag != "":
  params["contract_flag"] = contract_flag

res = requests.get(f"{API_URL}/derivatives/bars/daily/futures", params=params, headers=headers)

if res.status_code == 200:
  d = res.json()
  data = d["data"]
  while "pagination_key" in d:
    params["pagination_key"] = d["pagination_key"]
    res = requests.get(f"{API_URL}/derivatives/bars/daily/futures", params=params, headers=headers)
    d = res.json()
    data += d["data"]
  df = pd.DataFrame(data)
  display(df)
else:
  print(res.json())

In [ ]:
#@title オプション四本値（/derivatives/bars/daily/options）

#@markdown - オプション四本値APIでは、オプションに関する、四本値や清算値段、理論価格に関する情報を取得することができます。
#@markdown - データの取得では、日付（date）の指定が必須です。

#@markdown （データ更新時刻）
#@markdown - 毎営業日の27:00頃

category = ""#@param {type:"string"}
date = ""#@param {type:"string"}
contract_flag = ""#@param{type:"string"}

params = {}
if category != "":
  params["category"] = category
if date != "":
  params["date"] = date
if contract_flag != "":
  params["contract_flag"] = contract_flag

res = requests.get(f"{API_URL}/derivatives/bars/daily/options", params=params, headers=headers)

if res.status_code == 200:
  d = res.json()
  data = d["data"]
  while "pagination_key" in d:
    params["pagination_key"] = d["pagination_key"]
    res = requests.get(f"{API_URL}/derivatives/bars/daily/options", params=params, headers=headers)
    d = res.json()
    data += d["data"]
  df = pd.DataFrame(data)
  display(df)
else:
  print(res.json())

### Lightプラン以上 + アドオン契約で利用できるAPI
- 株価分足（/equities/bars/minute）

\* 株価 分足・ティック アドオンの契約が必要です。データ取得可能期間は過去2年間です。


In [ ]:
#@title 株価分足（/equities/bars/minute）

#@markdown - 分足の株価データを取得するエンドポイント。1分単位の四本値（始値・高値・安値・終値）、出来高、売買代金のデータを取得できます。
#@markdown - データの取得では、銘柄コード（code）または日付（date）の指定が必須となります。
#@markdown - データ取得可能期間は過去2年間です。

#@markdown （データ更新時刻）
#@markdown - 毎営業日の16:30頃

#@markdown （その他留意点）
#@markdown - 株価 分足・ティック アドオンの契約が必要です。
#@markdown - 全銘柄のデータを一括取得する場合は、Bulk APIの使用を推奨します。

code = ""#@param {type:"string"}
date = "20250122"#@param {type:"string"}
from_ = "" #@param {type:"string"}
to = "" #@param {type:"string"}

params = {}
if code != "":
  params["code"] = code
if date != "":
  params["date"] = date
if from_ != "":
  params["from"] = from_
if to != "":
  params["to"] = to

res = requests.get(f"{API_URL}/equities/bars/minute", params=params, headers=headers)

if res.status_code == 200:
  d = res.json()
  data = d["data"]
  while "pagination_key" in d:
    params["pagination_key"] = d["pagination_key"]
    res = requests.get(f"{API_URL}/equities/bars/minute", params=params, headers=headers)
    d = res.json()
    data += d["data"]
  df = pd.DataFrame(data)
  display(df)
else:
  print(res.json())


### Lightプラン以上で利用できるBulk API（CSVダウンロード）
- ダウンロード可能ファイル一覧（/bulk/list）
- ファイルダウンロード用URL取得（/bulk/get）

Bulk APIを使用すると、CSV形式で大量のデータを効率的にダウンロードできます。

\* FreeプランではCSV取得は利用できません。


In [ ]:
#@title ダウンロード可能ファイル一覧（/bulk/list）

#@markdown - CSV形式でダウンロード可能なファイルの一覧を取得するエンドポイントです。
#@markdown - endpointパラメータで取得したいデータセットを指定します。

#@markdown （指定可能なendpoint例）
#@markdown - /equities/bars/daily（株価四本値）
#@markdown - /equities/bars/minute（株価分足）
#@markdown - /fins/summary（財務情報）
#@markdown - /indices/bars/daily/topix（TOPIX指数四本値）
#@markdown - など

#@markdown 指定可能なendpointの完全な一覧は[こちら](https://jpx-jquants.com/ja/spec/bulk-list/endpoints)をご覧ください。

endpoint = "/equities/bars/daily"#@param {type:"string"}

params = {}
if endpoint != "":
  params["endpoint"] = endpoint

res = requests.get(f"{API_URL}/bulk/list", params=params, headers=headers)

if res.status_code == 200:
  d = res.json()
  data = d["data"]
  df = pd.DataFrame(data)
  display(df)
else:
  print(res.json())


In [ ]:
#@title ファイルダウンロード用URL取得（/bulk/get）

#@markdown - /bulk/list で取得したKeyを指定して、CSVファイルダウンロード用の署名付きURLを取得します。
#@markdown - 取得したURLの有効期限は約5分です。期限内にダウンロードを完了してください。

#@markdown （使い方）
#@markdown 1. まず /bulk/list でダウンロード可能なファイル一覧を取得
#@markdown 2. 取得したファイル一覧からダウンロードしたいファイルの Key を確認
#@markdown 3. その Key を下記に入力して実行
#@markdown 4. 取得したURLにアクセスしてCSVをダウンロード

key = ""#@param {type:"string"}

params = {}
if key != "":
  params["key"] = key

res = requests.get(f"{API_URL}/bulk/get", params=params, headers=headers)

if res.status_code == 200:
  d = res.json()
  download_url = d.get("url", "")
  print("ダウンロードURL（有効期限: 約5分）:")
  print(download_url)
  print("\n※ 上記URLにアクセスしてCSVファイルをダウンロードしてください。")
else:
  print(res.json())


In [ ]:
#@title Bulk APIを組み合わせた実行例（list → get）

#@markdown - /bulk/list でファイル一覧を取得し、特定のファイルの Key を使って /bulk/get でダウンロードURLを取得する実行例です。

#@markdown （使い方）
#@markdown 1. endpoint に取得したいデータセットのエンドポイントを指定
#@markdown 2. num_files に取得したいファイルの件数を指定（デフォルト: 3件）
#@markdown 3. 実行するとファイル一覧が表示されます
#@markdown 4. 一覧の中から指定された件数分のファイルの Key を取得し、ダウンロードURLを取得

endpoint = "/equities/bars/daily"#@param {type:"string"}
num_files = 3#@param {type:"integer"}

# Step 1: ファイル一覧を取得
params = {"endpoint": endpoint}
res = requests.get(f"{API_URL}/bulk/list", params=params, headers=headers)

if res.status_code == 200:
  d = res.json()
  files = d["data"]
  df_files = pd.DataFrame(files)
  
  print("=== 利用可能なファイル一覧 ===")
  display(df_files)
  
  if len(files) > 0:
    # 指定された件数分のファイルのKeyを取得（最大でファイル数まで）
    target_count = min(num_files, len(files))
    print(f"\n{target_count}件のファイルのダウンロードURLを取得します...")
    
    # Step 2: 各ファイルのダウンロードURLを取得
    for i in range(target_count):
      file_key = files[i]["Key"]
      print(f"\n--- ファイル {i+1}/{target_count} ---")
      print(f"Key: {file_key}")
      
      params_get = {"key": file_key}
      res_get = requests.get(f"{API_URL}/bulk/get", params=params_get, headers=headers)
      
      if res_get.status_code == 200:
        d_get = res_get.json()
        download_url = d_get.get("url", "")
        print(f"URL（有効期限: 約5分）:")
        print(download_url)
      else:
        print("ダウンロードURL取得エラー:")
        print(res_get.json())
    
    print("\n※ 上記URLにアクセスしてCSVファイルをダウンロードしてください。")
  else:
    print("\n利用可能なファイルがありません。")
else:
  print("ファイル一覧取得エラー:")
  print(res.json())